In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from load_datasets import load_strategyqa, load_gsm8k
from tqdm import tqdm
import re
import pickle
import numpy as np
from langchain.agents import initialize_agent
from copy import deepcopy
from langchain.agents import AgentType
from langchain.llms import OpenAI
import os

In [43]:
with open('VectorStore/embeddings.pkl', "rb") as f:
    docs = pickle.load(f)

In [4]:
print(len(docs.docs))

9251


In [5]:
np.unique([len(value['texts']) for key, value in docs.docs.items()])

array([1])

In [5]:
PATH = '../datasets/grade_school_math/data/train.jsonl'
train_data = load_gsm8k(PATH)


In [6]:
train_data[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

In [21]:
os.environ["BING_SUBSCRIPTION_KEY"] = "2075a41e8bd2447eae6595b3d4230ff8"
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

llm = OpenAI(temperature=0)
tools = [BingSearchTool(docs)]
self_ask_with_search_agent = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True,
                                              max_iterations=4)
my_agent = deepcopy(self_ask_with_search_agent)

with open('prompts/strategyqa_prompt_cot_new.txt', 'r') as file:
    content = file.read()

add_template = 'You are willing to answer questions that require reasoning. The final answer should be either "Yes" or "No". You must generate at least 2 follow up questions before generating the final answer. Follow the format of the examples below.\n'
fewshot = content.replace('Question', 'question').replace('Answer', 'answer')
preprocessed_fewshot = re.sub(r'Sub answer\s#\d+ :', 'Intermediate answer:', re.sub(r'Sub question\s#\d+ :', 'Follow up:', fewshot)).replace('question:', 'Question:').replace('Final answer:', 'So the final answer is:')
output_parser = 'Question: {input}\nOutput:{agent_scratchpad}'
my_agent.agent.llm_chain.prompt.template = add_template + preprocessed_fewshot + output_parser

In [22]:
print(my_agent.agent.llm_chain.prompt.template)

You are willing to answer questions that require reasoning. The final answer should be either "Yes" or "No". You must generate at least 2 follow up questions before generating the final answer. Follow the format of the examples below.

[Example 1]
Question: Do hamsters provide food for any animals?
Output:
Follow up: What type of animals are hamsters?
Intermediate answer: Hamsters are prey animals.
Follow up: Can prey animals be food for other animals?
Intermediate answer: Prey are food for predators.
So the final answer is: Since hamsters are prey animals, and prey are food for predetors, hamsters provide food for some animals, the answer is YES.

[Example 2]
Question: Could Brooke Shields succeed at University of Pennsylvania?
Output:
Follow up: What university did Brooke Shields went to?
Intermediate answer: Brooke Shields went to Princeton University.
Follow up: Did Brooke Shields succeed at Princeton University?
Intermediate answer: At Princeton University, she got all As and Bs w

In [19]:
i = 15
train_data[i]

{'qid': 'df84924c94e9e929ef10',
 'term': 'Nancy Pelosi',
 'description': '52nd speaker of the United States House of Representatives',
 'question': 'Would Nancy Pelosi publicly denounce abortion?',
 'answer': False,
 'facts': ['Abortion is a topic that divides the Republican Party and the Democratic Party.',
  'Nancy Pelosi is a Democrat and appeals to that base.',
  'The Democratic base is largely pro-choice with abortion. '],
 'decomposition': ['Which US political party does Nancy Pelosi identify with?',
  'Do most people who identify with #1 oppose abortion?'],
 'evidence': [[[['Nancy Pelosi-1']],
   [['Democratic Party (United States)-3'], 'operation']],
  [[['Nancy Pelosi-1']],
   [['Democratic Party (United States)-3'], 'no_evidence']],
  [[['Speaker of the United States House of Representatives-3']],
   [['Democratic Party (United States)-70']]]]}

In [23]:
query = train_data[i]['question']
answer = my_agent.run(query)
print(answer)



> Entering new AgentExecutor chain...

Follow up: What is Nancy Pelosi's political party?

Generated Search Query :             Nancy Pelosi AND political party
Indexing Page : Nancy_Pelosi__How_she_rose_to_the_top___and_stayed_there___BBC
Indexing Page : Nancy_Pelosi___Wikipedia
Indexing Page : Nancy_Pelosi__How_she_rose_to_the_top___and_stayed_there
Indexing Page : Nancy_Pelosi__39_s_Political_Party__Republican_or_Democrat____Celeb_Answers
Indexing Page : Nancy_Pelosi_s_political_philosophy__explained___Vox


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.



Intermediate answer: Question: What is Nancy Pelosi's political party?

Nancy Pelosi is a member of the Democratic Party (Nancy_Pelosi_2, Title2020).

References

1. (Titlea): Title: Nancy Pelosi&#39;s Political Party: Republican or Democrat? - Celeb Answers - Link: https://celebanswers.com/nancy-pelosis-political-party-republican-or-democrat/

2. (Nancy_Pelosi_12): Nancy_Pelosi_12

3. (Nancy_Pelosi_2): Nancy_Pelosi_2

4. (Title2020): Title: Nancy Pelosi’s political philosophy, explained - Vox - Link: https://www.vox.com/policy-and-politics/2020/5/15/21247994/nancy-pelosi-democratic-party-molly-ball

5. (Title): Title: Nancy Pelosi - Wikipedia - Link: https://en.wikipedia.org/wiki/Nancy_Pelosi

Tokens Used: 9541 Cost: $0.02

Follow up: What is Nancy Pelosi's stance on abortion?

Generated Search Query : Nancy Pelosi AND abortion
Indexing Page : WATCH__Speaker_Nancy_Pelosi_says_House_will_vote_to_codify_abortion____
Indexing Page : Nancy_Pelosi_meets_Pope_Francis_in_Rome_as_abortion_de

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.



Intermediate answer: Question: What is Nancy Pelosi's stance on abortion?

Nancy Pelosi is a staunch supporter of abortion rights (Titlec). She has voted against the Partial-Birth Abortion Ban Act of 2003 and earlier attempts at similar bans, and voted against the criminalization of certain situations where a minor is transported across state lines for an abortion (Title). Pelosi has voted in favor of lifting the ban on privately funded abortions at overseas U.S. military facilities, in favor of an amendment that would repeal a provision forbidding servicewomen and dependents from getting an abortion in overseas military hospitals, and in favor of stripping the prohibition of funding for organizations working overseas that use their own funds to provide abortion services, or engage in advocacy related to abortion services (Title). She supports legal abortion and has brought the Women's Health Protection Act, which would override state abortion laws and allow for abortions in some case

In [8]:
import pickle 

with open('VectorStore/embeddings.pkl', "rb") as f:
    docs = pickle.load(f)

docs

In [45]:
from paperqa_tool import PaperQATool

In [46]:
tool = PaperQATool(docs)

In [47]:
os.environ["BING_SUBSCRIPTION_KEY"] = "2075a41e8bd2447eae6595b3d4230ff8"
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

In [28]:
answer = tool._run('Where do harbor seals usually live?')



Generated Search Query : harbor seals AND habitat
This Page already in the Docs
This Page already in the Docs
This Page already in the Docs
This Page already in the Docs

Answer:
 Question: Where do harbor seals usually live?

Harbor seals tend to be coastal and do not venture more than 20 km offshore. They may spend several days at sea and travel up to 50 km in search of feeding grounds. Resting sites may be both rugged, rocky coasts or sandy beaches. They frequently congregate in harbors, bays, sandy intertidal zones, and estuaries in pursuit of prey fish. They occasionally swim more than a hundred miles upstream into fresh water in large rivers in search of migratory fish like shad and likely salmon (Titlec). However, the provided context from the National Zoological Park__United_States__26 does not provide any additional information on where harbor seals usually live.

References

1. (Titlec): Title: Harbor seal - Wikipedia - Link: https://en.wikipedia.org/wiki/Harbor_seal

2. (N

In [29]:
print(answer.formatted_answer)

Question: Where do harbor seals usually live?

Harbor seals tend to be coastal and do not venture more than 20 km offshore. They may spend several days at sea and travel up to 50 km in search of feeding grounds. Resting sites may be both rugged, rocky coasts or sandy beaches. They frequently congregate in harbors, bays, sandy intertidal zones, and estuaries in pursuit of prey fish. They occasionally swim more than a hundred miles upstream into fresh water in large rivers in search of migratory fish like shad and likely salmon (Titlec). However, the provided context from the National Zoological Park__United_States__26 does not provide any additional information on where harbor seals usually live.

References

1. (Titlec): Title: Harbor seal - Wikipedia - Link: https://en.wikipedia.org/wiki/Harbor_seal

2. (National_Zoological_Park__United_States__26): National_Zoological_Park__United_States__26

Tokens Used: 5676 Cost: $0.01


In [30]:
len(answer.contexts)

2

In [31]:
answer.context

'Titlec: Harbor seals tend to be coastal, not venturing more than 20 km offshore. They may spend several days at sea and travel up to 50 km in search of feeding grounds. Resting sites may be both rugged, rocky coasts or sandy beaches. They frequently congregate in harbors, bays, sandy intertidal zones, and estuaries in pursuit of prey fish. They occasionally swim more than a hundred miles upstream into fresh water in large rivers in search of migratory fish like shad and likely salmon.\n\nNational_Zoological_Park__United_States__26: The text mentions that harbor seals are one of the species housed in the American Trail exhibit at the National Zoological Park. However, it does not provide information on where they usually live.\n\nValid keys: Titlec, National_Zoological_Park__United_States__26'

In [33]:
answer.contexts[0][2]

'Harbor seals tend to be coastal, not venturing more than 20 km offshore. They may spend several days at sea and travel up to 50 km in search of feeding grounds. Resting sites may be both rugged, rocky coasts or sandy beaches. They frequently congregate in harbors, bays, sandy intertidal zones, and estuaries in pursuit of prey fish. They occasionally swim more than a hundred miles upstream into fresh water in large rivers in search of migratory fish like shad and likely salmon.'

In [36]:
print(answer.context)

Titlec: Harbor seals tend to be coastal, not venturing more than 20 km offshore. They may spend several days at sea and travel up to 50 km in search of feeding grounds. Resting sites may be both rugged, rocky coasts or sandy beaches. They frequently congregate in harbors, bays, sandy intertidal zones, and estuaries in pursuit of prey fish. They occasionally swim more than a hundred miles upstream into fresh water in large rivers in search of migratory fish like shad and likely salmon.

National_Zoological_Park__United_States__26: The text mentions that harbor seals are one of the species housed in the American Trail exhibit at the National Zoological Park. However, it does not provide information on where they usually live.

Valid keys: Titlec, National_Zoological_Park__United_States__26


In [38]:
answer.contexts[0][2]

'Harbor seals tend to be coastal, not venturing more than 20 km offshore. They may spend several days at sea and travel up to 50 km in search of feeding grounds. Resting sites may be both rugged, rocky coasts or sandy beaches. They frequently congregate in harbors, bays, sandy intertidal zones, and estuaries in pursuit of prey fish. They occasionally swim more than a hundred miles upstream into fresh water in large rivers in search of migratory fish like shad and likely salmon.'

In [39]:
answer.contexts[1][2]

'The text mentions that harbor seals are one of the species housed in the American Trail exhibit at the National Zoological Park. However, it does not provide information on where they usually live.'

In [49]:
answer.contexts[0]

('Titlec',
 'Title: Harbor seal - Wikipedia - Link: https://en.wikipedia.org/wiki/Harbor_seal',
 'Harbor seals tend to be coastal, not venturing more than 20 km offshore. They may spend several days at sea and travel up to 50 km in search of feeding grounds. Resting sites may be both rugged, rocky coasts or sandy beaches. They frequently congregate in harbors, bays, sandy intertidal zones, and estuaries in pursuit of prey fish. They occasionally swim more than a hundred miles upstream into fresh water in large rivers in search of migratory fish like shad and likely salmon.',
 '\\)"), 1942) | [Ungava seals](/wiki/Ungava_seal\n"Ungava seal") ([Lacs des Loups Marins](/wiki/Lacs_des_Loups_Marins "Lacs des\nLoups Marins") seals) | found in eastern Canada in [fresh\nwater](/wiki/Freshwater_seal "Freshwater seal")  \n[![Harbor Seal with young\npup.jpg](//upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Harbor_Seal_with_young_pup.jpg/120px-\nHarbor_Seal_with_young_pup.jpg)](/wiki/File:Harbor_

In [50]:
answer.contexts[1]

('National_Zoological_Park__United_States__26',
 'National_Zoological_Park__United_States__26',
 'The text mentions that harbor seals are one of the species housed in the American Trail exhibit at the National Zoological Park. However, it does not provide information on where they usually live.',
 'The American Trail exhibit houses a variety of North American species. These include California sea lions, grey seals, harbor seals, North American beavers, North American river otters, bald eagles, common ravens, brown pelicans, and wolves. After facing severe threats, the majority of American Trail species have rebounded thanks to conservation efforts. Many of the residents of American Trail have been listed as endangered. All of plants in the animal enclosures on American Trail exhibit are native to North America.')